# LCEL

In [12]:
import os
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv()

import google.generativeai as genai
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY


In [5]:
from langchain_core.runnables import RunnableLambda

def add_five(x):
    return x+5

def multiply_by_two(x):
    return x*2

add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)
chain = add_five | multiply_by_two
chain.invoke(5) # 5+5 = 10 -> 10*2 = 20

20

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

llm = GoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)

prompt = PromptTemplate(
    input_variables=["topic","style"],
     template= "Tell me a short story about {topic} in {style} style"
)

story_chain = prompt | llm | StrOutputParser()

response = story_chain.invoke({
    "topic": "a brave astronaut",
    "style": "science fiction"
})

print("Story Result:", response)

Story Result: Captain Elara Vance, her face a roadmap of wrinkles etched by solar flares and zero-g, stared out at Xylos. The planet, a swirling turquoise and emerald behemoth, pulsed with a malevolent energy.  Xylos was known as the Graveyard of Starships – a beautiful death trap.  Her mission: retrieve the black box from the *Stardust*, a vessel swallowed by Xylos's unpredictable gravitational tides a decade ago.  No one had returned.

Elara wasn't afraid. Fear was a luxury she'd shed years ago, orbiting Jupiter's storm clouds.  She strapped herself into the *Sparrow*, her nimble, single-person exploration craft, its hull shimmering with a newly developed anti-graviton shield.  Xylos's gravity was a chaotic beast, tearing at the fabric of spacetime.  Even the Sparrow shuddered as it plunged into the turbulent atmosphere.

The *Stardust*, half-buried in a crystalline forest that shimmered with an unnerving bioluminescence, was a mangled wreck.  Elara navigated the treacherous terrain,

### Function Calling

In [20]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel , Field

class BookRecommendation(BaseModel):
    title : str = Field(description="The Book Title")
    topic : str = Field(description="Topic of the book")
    author : str = Field(description="Author of the Book")
    reason : str = Field(description="Why this Book is recommended")


book_parser = PydanticOutputParser(pydantic_object=BookRecommendation)

book_prompt = PromptTemplate(input_variables=["Topic","format_instructions"],
                        template="Recommend a Book based on the following preferences Topic : {Topic} {format_instructions} ")

book_chain = book_prompt | llm | book_parser

response = book_chain.invoke({"Topic":"Large Language Model","format_instructions": book_parser.get_format_instructions()})
print("Recommended Book :",response)

Recommended Book : title='Deep Learning with Python' topic='Large Language Model' author='François Chollet' reason='While not exclusively focused on LLMs, this book provides a strong foundation in deep learning principles, which are crucial for understanding the inner workings of large language models.  It covers relevant topics like recurrent neural networks and attention mechanisms, essential components of LLMs.  The practical examples and clear explanations make it accessible to a wide audience.'
